In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
'''
!pip install workalendar
!pip install lightgbm==3.3.2
!pip install category_encoders
'''

'\n!pip install workalendar\n!pip install lightgbm==3.3.2\n!pip install category_encoders\n'

In [53]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

from pandas import Timestamp, DateOffset
from workalendar.asia import SouthKorea

# matplotlib 폰트설정
# plt.rc('font', family='NanumGothicOTF') # For MacOS
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

In [54]:
train=pd.read_csv("/content/drive/MyDrive/BITAmin/DaconData/train.csv")
test=pd.read_csv("/content/drive/MyDrive/BITAmin/DaconData/test.csv")
ss=pd.read_csv("/content/drive/MyDrive/BITAmin/DaconData/sample_submission.csv")
acc=pd.read_csv("/content/drive/MyDrive/BITAmin/DaconData/external_open/countrywide_accident.csv")

In [55]:
#이상치 제거
train=train[train["ECLO"]<=20]
acc=acc[acc["ECLO"]<=20]

In [56]:
#데이터 분리
train1=train[train["사고유형"]=="차량단독"]
train2=train[train["사고유형"]=="차대차"]
train3=train[train["사고유형"]=="차대사람"]

test1=test[test["사고유형"]=="차량단독"]
test2=test[test["사고유형"]=="차대차"]
test3=test[test["사고유형"]=="차대사람"]

In [57]:
all_car_alone=acc[acc["사고유형"]=="차량단독"]
c_to_c=acc[acc["사고유형"]=="차대차"]
c_to_h=acc[acc["사고유형"]=="차대사람"]
train1=pd.concat([train1,all_car_alone])
train2=pd.concat([train2,c_to_c])
train3=pd.concat([train3,c_to_h])

In [58]:
# target 변수 지정
y_train1 = train1['ECLO']
y_train2 = train2['ECLO']
y_train3 = train3['ECLO']
# train, test 데이터간 컬럼 동기화
cols = test1.columns
X_train1 = train1[cols]
X_test1 = test1

X_train2 = train2[cols]
X_test2 = test2

X_train3 = train3[cols]

X_test3 = test3

In [59]:
def feat_eng1(df):
  # == ID 패스 ==

  # == 사고일시 ==
  time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})'

  df[['연', '월', '일', '시간']] = df['사고일시'].str.extract(time_pattern)
  df[['연', '월', '일', '시간']] = df[['연', '월', '일', '시간']].apply(pd.to_numeric) # 추출된 문자열을 수치화해줍니다

  # = 공휴일 column 추가 =
  def is_holiday(year, month, day):
    cal = SouthKorea()
    return cal.is_holiday(Timestamp(year, month, day))

  def classify_day(year, month, day):
    date = Timestamp(year, month, day)
    if date.dayofweek < 5 and not is_holiday(year, month, day):
      return 0
    else:
      return 1

  df['Holiday'] = df.apply(lambda row: classify_day(row['연'], row['월'], row['일']), axis=1)
  df.drop(columns=['사고일시','일','연','요일'], inplace = True) # 정보 추출이 완료된 '사고일시','일','요일','연' 컬럼은 제거합니다


  # == 봄, 여름, 가을, 겨울, 기상학적으로 구분함 ==
  def categorize_season(month):
    if month in [3, 4, 5]:
      return '봄'

    elif month in [6, 7, 8]:
      return '여름'

    elif month in [9, 10, 11]:
      return '가을'

    else:
      return '겨울'

  df['계절'] = df['월'].apply(categorize_season)

  # == 출 퇴근 시간(공휴일까지 고려) ==  평일에는 7~9, 18~20시까지 출퇴근, 공휴일에는 18~22시까지 교통량 증가로 확인
  def rush_hour(hour, holiday):
      if holiday == 1 or (holiday == 0 and hour in [18, 19, 20, 21, 22]):
          return "출퇴근"
      else:
          return "노출퇴근"

  df['출퇴근'] = df.apply(lambda row: rush_hour(row['시간'], row['Holiday']), axis=1)


  # == 시군구, 도시는 버리지 않음 ==
  location_pattern = r'(\S+) (\S+) (\S+)'

  df[['도시', '구', '동']] = df['시군구'].str.extract(location_pattern)
  df.drop(columns=['시군구', '도시', '사고유형'], inplace = True)

  # == 도로형태 == -> 분리하지말고 target encoding

  # == 기상상태, 노면상태, 계절 -> 원핫인코딩 ==
  df = pd.get_dummies(df, columns = ['기상상태', '노면상태', '계절', '출퇴근'])

  return df

In [60]:
def feat_eng2(df):
  # == ID 패스 ==

  # == 사고일시 ==
  time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})'

  df[['연', '월', '일', '시간']] = df['사고일시'].str.extract(time_pattern)
  df[['연', '월', '일', '시간']] = df[['연', '월', '일', '시간']].apply(pd.to_numeric) # 추출된 문자열을 수치화해줍니다

  # = 공휴일 column 추가 =
  def is_holiday(year, month, day):
    cal = SouthKorea()
    return cal.is_holiday(Timestamp(year, month, day))

  def classify_day(year, month, day):
    date = Timestamp(year, month, day)
    if date.dayofweek < 5 and not is_holiday(year, month, day):
      return 0
    else:
      return 1

  df['Holiday'] = df.apply(lambda row: classify_day(row['연'], row['월'], row['일']), axis=1)
  df.drop(columns=['사고일시','일','연','요일'], inplace = True) # 정보 추출이 완료된 '사고일시','일','요일','연' 컬럼은 제거합니다


  # == 봄, 여름, 가을, 겨울, 기상학적으로 구분함 ==
  def categorize_season(month):
    if month in [3, 4, 5]:
      return '봄'

    elif month in [6, 7, 8]:
      return '여름'

    elif month in [9, 10, 11]:
      return '가을'

    else:
      return '겨울'

  df['계절'] = df['월'].apply(categorize_season)

  # == 출 퇴근 시간(공휴일까지 고려) ==  평일에는 7~9, 18~20시까지 출퇴근, 공휴일에는 18~22시까지 교통량 증가로 확인
  def rush_hour(hour, holiday):
      if holiday == 1 or (holiday == 0 and hour in [18, 19, 20, 21, 22]):
          return "출퇴근"
      else:
          return "노출퇴근"

  df['출퇴근'] = df.apply(lambda row: rush_hour(row['시간'], row['Holiday']), axis=1)


  # == 시군구, 도시는 버리지 않음 ==
  location_pattern = r'(\S+) (\S+) (\S+)'

  df[['도시', '구', '동']] = df['시군구'].str.extract(location_pattern)
  df.drop(columns=['시군구', '도시', '사고유형'], inplace = True)

  # == 도로형태 == -> 분리하지말고 target encoding

  # == 기상상태, 노면상태, 계절 -> 원핫인코딩 ==
  df = pd.get_dummies(df, columns = ['기상상태', '노면상태', '계절', '출퇴근'])

  return df

In [61]:
def feat_eng3(df):
  # == ID 패스 ==

  # == 사고일시 ==
  time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})'

  df[['연', '월', '일', '시간']] = df['사고일시'].str.extract(time_pattern)
  df[['연', '월', '일', '시간']] = df[['연', '월', '일', '시간']].apply(pd.to_numeric) # 추출된 문자열을 수치화해줍니다

  # = 공휴일 column 추가 =
  def is_holiday(year, month, day):
    cal = SouthKorea()
    return cal.is_holiday(Timestamp(year, month, day))

  def classify_day(year, month, day):
    date = Timestamp(year, month, day)
    if date.dayofweek < 5 and not is_holiday(year, month, day):
      return 0
    else:
      return 1

  df['Holiday'] = df.apply(lambda row: classify_day(row['연'], row['월'], row['일']), axis=1)
  df.drop(columns=['사고일시','일','연','요일'], inplace = True) # 정보 추출이 완료된 '사고일시','일','요일','연' 컬럼은 제거합니다


  # == 봄, 여름, 가을, 겨울, 기상학적으로 구분함 ==
  def categorize_season(month):
    if month in [3, 4, 5]:
      return '봄'

    elif month in [6, 7, 8]:
      return '여름'

    elif month in [9, 10, 11]:
      return '가을'

    else:
      return '겨울'

  df['계절'] = df['월'].apply(categorize_season)

  # == 출 퇴근 시간(공휴일까지 고려) ==  평일에는 7~9, 18~20시까지 출퇴근, 공휴일에는 18~22시까지 교통량 증가로 확인
  def rush_hour(hour, holiday):
      if holiday == 1 or (holiday == 0 and hour in [18, 19, 20, 21, 22]):
          return "출퇴근"
      else:
          return "노출퇴근"

  df['출퇴근'] = df.apply(lambda row: rush_hour(row['시간'], row['Holiday']), axis=1)


  # == 시군구, 도시는 버리지 않음 ==
  location_pattern = r'(\S+) (\S+) (\S+)'

  df[['도시', '구', '동']] = df['시군구'].str.extract(location_pattern)
  df.drop(columns=['시군구', '도시', '사고유형'], inplace = True)

  # == 도로형태 == -> 분리하지말고 target encoding

  # == 기상상태, 노면상태, 계절 -> 원핫인코딩 ==
  df = pd.get_dummies(df, columns = ['기상상태', '노면상태', '계절', '출퇴근'])

  return df

In [62]:
X_train_eng1 = feat_eng1(X_train1)
X_test_eng1 = feat_eng1(X_test1)

In [63]:
X_train_eng2 = feat_eng2(X_train2)
X_test_eng2 = feat_eng2(X_test2)

In [64]:
X_train_eng3 = feat_eng3(X_train3)
X_test_eng3 = feat_eng3(X_test3)

In [65]:
list1=[]
list2=[]
list3=[]
del_ID1=[]
del_ID2=[]
del_ID3=[]
for i in X_train_eng1.columns:
    if i not in X_test_eng1.columns :
        list1.append(i)
for i in list1:
    for j in X_train_eng1[X_train_eng1[i]==1]["ID"] :
        del_ID1.append(j)
    X_train_eng1=X_train_eng1[X_train_eng1[i]==0].drop(i, axis=1)
X_train_eng1.sort_index(axis=1,inplace=True)

for i in X_train_eng2.columns:
    if i not in X_test_eng2.columns:
        list2.append(i)
for i in list2:
    for j in X_train_eng2[X_train_eng2[i]==1]["ID"] :
        del_ID2.append(j)
    X_train_eng2=X_train_eng2[X_train_eng2[i]==0].drop(i, axis=1)
X_train_eng2.sort_index(axis=1,inplace=True)

for i in X_train_eng3.columns:
    if i not in X_test_eng3.columns:
        list3.append(i)
for i in list3:
    for j in X_train_eng3[X_train_eng3[i]==1]["ID"] :
        del_ID3.append(j)
    X_train_eng3=X_train_eng3[X_train_eng3[i]==0].drop(i, axis=1)
X_train_eng3.sort_index(axis=1,inplace=True)

In [66]:
X_train_eng1.drop("ID", axis=1, inplace=True)
X_train_eng2.drop("ID", axis=1, inplace=True)
X_train_eng3.drop("ID", axis=1, inplace=True)

In [67]:
for i in X_test_eng1.columns:
    if i not in X_train_eng1.columns:
        X_test_eng1.drop(i, axis=1, inplace=True)
X_test_eng1.sort_index(axis=1, inplace=True)

for i in X_test_eng2.columns:
    if i not in X_train_eng2.columns:
        X_test_eng2.drop(i, axis=1, inplace=True)
X_test_eng2.sort_index(axis=1, inplace=True)

for i in X_test_eng3.columns:
    if i not in X_train_eng3.columns:
        X_test_eng3.drop(i, axis=1, inplace=True)
X_test_eng3.sort_index(axis=1, inplace=True)

In [68]:
def rmsle(y,pred) :
    log_y=np.log1p(y)
    log_pred=np.log1p(pred)
    squared_error = (log_y - log_pred) ** 2
    rmsle=np.sqrt(np.mean(squared_error))
    return rmsle

In [69]:
y_train1 = train1[~(train1["ID"].isin(del_ID1))]['ECLO']
y_train2 = train2[~(train2["ID"].isin(del_ID2))]['ECLO']
y_train3 = train3[~(train3["ID"].isin(del_ID3))]['ECLO']

In [70]:
from category_encoders.target_encoder import TargetEncoder

#
categorical_features = list(X_train_eng1.dtypes[X_train_eng1.dtypes == "object"].index)
# categorical_features.append("월")
# categorical_features.append("시")
# 추출된 문자열 변수 확인
display(categorical_features)

for i in categorical_features:
    le = TargetEncoder(cols=[i])
    X_train_eng1[i] = le.fit_transform(X_train_eng1[i], y_train1)
    X_test_eng1[i] = le.transform(X_test_eng1[i])

categorical_features = list(X_train_eng2.dtypes[X_train_eng2.dtypes == "object"].index)
# categorical_features.append("월")
# categorical_features.append("시")
# 추출된 문자열 변수 확인
display(categorical_features)

for i in categorical_features:
    le = TargetEncoder(cols=[i])
    X_train_eng2[i] = le.fit_transform(X_train_eng2[i], y_train2)
    X_test_eng2[i] = le.transform(X_test_eng2[i])

categorical_features = list(X_train_eng3.dtypes[X_train_eng3.dtypes == "object"].index)
# 추출된 문자열 변수 확인
display(categorical_features)
# categorical_features.append("월")
# categorical_features.append("시")
for i in categorical_features:
    le = TargetEncoder(cols=[i])
    X_train_eng3[i] = le.fit_transform(X_train_eng3[i], y_train3)
    X_test_eng3[i] = le.transform(X_test_eng3[i])

['구', '도로형태', '동']

['구', '도로형태', '동']

['구', '도로형태', '동']

In [71]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import numpy as np

def rmsle(y, pred):
    log_y = np.log1p(y)
    log_pred = np.log1p(pred)
    squared_error = (log_y - log_pred) ** 2
    rmsle = np.sqrt(np.mean(squared_error))
    return rmsle

def custom_rmsle(y_true, y_pred):
    rmsle_value = float(rmsle(y_true, y_pred))
    return 'RMSLE', rmsle_value, False

# Log transformation of target variable
y_train_log_total1 = np.log1p(y_train1)
X_train1, X_test1, y_train_log1, y_test_log1 = train_test_split(X_train_eng1, y_train_log_total1, test_size=0.2, random_state=42)
X_tr1, X_val1, y_tr_log1, y_val_log1 = train_test_split(X_train1, y_train_log1, test_size=0.3, random_state=42)

y_train_log_total2 = np.log1p(y_train2)
X_train2, X_test2, y_train_log2, y_test_log2 = train_test_split(X_train_eng2, y_train_log_total2, test_size=0.2, random_state=42)
X_tr2, X_val2, y_tr_log2, y_val_log2 = train_test_split(X_train2, y_train_log2, test_size=0.3, random_state=42)

y_train_log_total3 = np.log1p(y_train3)
X_train3, X_test3, y_train_log3, y_test_log3 = train_test_split(X_train_eng3, y_train_log_total3, test_size=0.2, random_state=42)
X_tr3, X_val3, y_tr_log3, y_val_log3 = train_test_split(X_train3, y_train_log3, test_size=0.3, random_state=42)


In [ ]:
from sklearn.model_selection import KFold
# Define the objective function for hyperopt
def objective(params):
    lgbm_params = {
        'learning_rate': params['learning_rate'],
        'n_estimators': int(params['n_estimators']),
        'num_leaves': int(params['num_leaves']),
        'max_depth': int(params['max_depth']),
        'subsample': params['subsample'],
        'colsample_bytree': params['colsample_bytree'],
        'min_child_samples': int(params['min_child_samples']),
        'reg_alpha': params['reg_alpha'],
        'reg_lambda': params['reg_lambda'],
        'min_split_gain': params['min_split_gain']
    }
    lgbm_model = LGBMRegressor(**lgbm_params, random_state=0, n_jobs = -1)

    # 3개의 k-fold 방식으로 평가된 rmsle 지표를 담는 list
    rmsle_list = []

    # 3-fold k-fold 방식 적용
    kf = KFold(n_splits=3)
    for tr_index, val_index in kf.split(X_train1):
        # kf.split(X_train)으로 추출된 학습과 검증 index값으로 학습과 검증 데이터 세트 분리
        X_tr, y_log_tr = X_train1.iloc[tr_index], y_train_log_total1.iloc[tr_index]
        X_val, y_log_val = X_train1.iloc[val_index], y_train_log_total1.iloc[val_index]

        # early stopping은 30회로 설정하고 추출된 학습과 검증 데이터로 XGBClassifier 학습 수행.
        lgbm_model.fit(X_tr, y_log_tr, early_stopping_rounds=30, eval_metric= custom_rmsle,
                       eval_set=[(X_tr, y_log_tr), (X_val, y_log_val)])
        y_val_pred = np.expm1(lgbm_model.predict(X_val))
        rmsle_val = rmsle(y_val_pred, np.expm1(y_log_val))
        rmsle_list.append(rmsle_val)

    # Return the mean rmsle as the loss
    return np.mean(rmsle_list)



# Define the search space for hyperparameters
space1 = {
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'n_estimators': hp.quniform('n_estimators', 50, 500, 1),
    'num_leaves': hp.quniform('num_leaves', 10, 100, 1),
    'max_depth': hp.quniform('max_depth', 5, 15, 1),
    'subsample': hp.uniform('subsample', 0.7, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.7, 1),
    'min_child_samples': hp.quniform('min_child_samples', 20, 100, 1),
    'reg_alpha': hp.loguniform('reg_alpha', np.log(1e-9), np.log(1)),
    'reg_lambda': hp.loguniform('reg_lambda', np.log(1e-9), np.log(1)),
    'min_split_gain': hp.loguniform('min_split_gain', np.log(1e-9), np.log(1))
}

# Create Trials object to store optimization results
trials = Trials()

# Run hyperopt optimization
best1 = fmin(fn=objective, space=space1, algo=tpe.suggest, max_evals=100, trials=trials, rstate=np.random.default_rng(seed=30))
print(best1)

best_params1 = {
    'learning_rate': best1["learning_rate"],
    'n_estimators': int(best1['n_estimators']),
    'num_leaves': int(best1['num_leaves']),
    'max_depth': int(best1['max_depth']),
    'min_child_samples': int(best1['min_child_samples']),
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'subsample': best1['subsample'],
    'colsample_bytree': best1['colsample_bytree'],
    'reg_alpha': best1['reg_alpha'],
    'reg_lambda': best1['reg_lambda'],
    'min_split_gain': best1['min_split_gain']
}

# Print the best hyperparameters
print("Best Hyperparameters:")
print(best_params1)


In [ ]:
lg1 = LGBMRegressor(**best_params1, random_state=0, n_jobs = -1)

lg1.fit(X_tr1, y_tr_log1, early_stopping_rounds=100, eval_metric=custom_rmsle, eval_set=[(X_tr1, y_tr_log1), (X_val1, y_val_log1)])

y_test_pred_log1 = lg1.predict(X_test1)
y_test_pred1 = np.expm1(y_test_pred_log1)

rmsle_test = rmsle(np.expm1(y_test_log1), y_test_pred1)
print('Test RMSLE:', rmsle_test)
#Test RMSLE: 0.4747587094525547

In [74]:
X_train_eng2.shape

(493672, 23)

In [ ]:
from sklearn.model_selection import KFold
# Define the objective function for hyperopt
def objective(params):
    lgbm_params = {
        'learning_rate': params['learning_rate'],
        'n_estimators': int(params['n_estimators']),
        'num_leaves': int(params['num_leaves']),
        'max_depth': int(params['max_depth']),
        'subsample': params['subsample'],
        'colsample_bytree': params['colsample_bytree'],
        'min_child_samples': int(params['min_child_samples']),
        'reg_alpha': params['reg_alpha'],
        'reg_lambda': params['reg_lambda'],
        'min_split_gain': params['min_split_gain']
    }
    lgbm_model = LGBMRegressor(**lgbm_params, random_state=0, n_jobs = -1)

    # 3개의 k-fold 방식으로 평가된 rmsle 지표를 담는 list
    rmsle_list = []

    # 3-fold k-fold 방식 적용
    kf = KFold(n_splits=3)
    for tr_index, val_index in kf.split(X_train2):
        # kf.split(X_train)으로 추출된 학습과 검증 index값으로 학습과 검증 데이터 세트 분리
        X_tr, y_log_tr = X_train2.iloc[tr_index], y_train_log_total2.iloc[tr_index]
        X_val, y_log_val = X_train2.iloc[val_index], y_train_log_total2.iloc[val_index]

        # early stopping은 30회로 설정하고 추출된 학습과 검증 데이터로 XGBClassifier 학습 수행.
        lgbm_model.fit(X_tr, y_log_tr, early_stopping_rounds=30, eval_metric= custom_rmsle,
                       eval_set=[(X_tr, y_log_tr), (X_val, y_log_val)])
        y_val_pred = np.expm1(lgbm_model.predict(X_val))
        rmsle_val = rmsle(y_val_pred, np.expm1(y_log_val))
        rmsle_list.append(rmsle_val)

    # Return the mean rmsle as the loss
    return np.mean(rmsle_list)



# Define the search space for hyperparameters
space2 = {
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'n_estimators': hp.quniform('n_estimators', 50, 1000, 1),
    'num_leaves': hp.quniform('num_leaves', 30, 200, 1),
    'max_depth': hp.quniform('max_depth', 5, 50, 1),
    'subsample': hp.uniform('subsample', 0.7, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.7, 1),
    'min_child_samples': hp.quniform('min_child_samples', 20, 200, 1),
    'reg_alpha': hp.loguniform('reg_alpha', np.log(1e-9), np.log(1)),
    'reg_lambda': hp.loguniform('reg_lambda', np.log(1e-9), np.log(1)),
    'min_split_gain': hp.loguniform('min_split_gain', np.log(1e-9), np.log(1))
}

# Create Trials object to store optimization results
trials = Trials()

# Run hyperopt optimization
best2 = fmin(fn=objective, space=space2, algo=tpe.suggest, max_evals=200, trials=trials, rstate=np.random.default_rng(seed=30))
print(best2)

best_params2 = {
    'learning_rate': best2["learning_rate"],
    'n_estimators': int(best2['n_estimators']),
    'num_leaves': int(best2['num_leaves']),
    'max_depth': int(best2['max_depth']),
    'min_child_samples': int(best2['min_child_samples']),
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'subsample': best2['subsample'],
    'colsample_bytree': best2['colsample_bytree'],
    'reg_alpha': best2['reg_alpha'],
    'reg_lambda': best2['reg_lambda'],
    'min_split_gain': best2['min_split_gain']
}

# Print the best hyperparameters
print("Best Hyperparameters:")
print(best_params2)


In [ ]:
lg2 = LGBMRegressor(**best_params2, random_state=0, n_jobs = -1)

lg2.fit(X_tr2, y_tr_log2, early_stopping_rounds=100, eval_metric=custom_rmsle, eval_set=[(X_tr2, y_tr_log2), (X_val2, y_val_log2)])

y_test_pred_log2 = lg2.predict(X_test2)
y_test_pred2 = np.expm1(y_test_pred_log2)

rmsle_test = rmsle(np.expm1(y_test_log2), y_test_pred2)
print('Test RMSLE:', rmsle_test)
#Test RMSLE: 0.4747587094525547

In [77]:
X_train_eng3.shape

(117176, 20)

In [ ]:
from sklearn.model_selection import KFold
# Define the objective function for hyperopt
def objective(params):
    lgbm_params = {
        'learning_rate': params['learning_rate'],
        'n_estimators': int(params['n_estimators']),
        'num_leaves': int(params['num_leaves']),
        'max_depth': int(params['max_depth']),
        'subsample': params['subsample'],
        'colsample_bytree': params['colsample_bytree'],
        'min_child_samples': int(params['min_child_samples']),
        'reg_alpha': params['reg_alpha'],
        'reg_lambda': params['reg_lambda'],
        'min_split_gain': params['min_split_gain']
    }
    lgbm_model = LGBMRegressor(**lgbm_params, random_state=0, n_jobs = -1)

    # 3개의 k-fold 방식으로 평가된 rmsle 지표를 담는 list
    rmsle_list = []

    # 3-fold k-fold 방식 적용
    kf = KFold(n_splits=3)
    for tr_index, val_index in kf.split(X_train3):
        # kf.split(X_train)으로 추출된 학습과 검증 index값으로 학습과 검증 데이터 세트 분리
        X_tr, y_log_tr = X_train3.iloc[tr_index], y_train_log_total3.iloc[tr_index]
        X_val, y_log_val = X_train3.iloc[val_index], y_train_log_total3.iloc[val_index]

        # early stopping은 30회로 설정하고 추출된 학습과 검증 데이터로 XGBClassifier 학습 수행.
        lgbm_model.fit(X_tr, y_log_tr, early_stopping_rounds=30, eval_metric= custom_rmsle,
                       eval_set=[(X_tr, y_log_tr), (X_val, y_log_val)])
        y_val_pred = np.expm1(lgbm_model.predict(X_val))
        rmsle_val = rmsle(y_val_pred, np.expm1(y_log_val))
        rmsle_list.append(rmsle_val)

    # Return the mean rmsle as the loss
    return np.mean(rmsle_list)



# Define the search space for hyperparameters
space3 = {
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'n_estimators': hp.quniform('n_estimators', 50, 750, 1),
    'num_leaves': hp.quniform('num_leaves', 20, 150, 1),
    'max_depth': hp.quniform('max_depth', 5, 30, 1),
    'subsample': hp.uniform('subsample', 0.7, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.7, 1),
    'min_child_samples': hp.quniform('min_child_samples', 20, 150, 1),
    'reg_alpha': hp.loguniform('reg_alpha', np.log(1e-9), np.log(1)),
    'reg_lambda': hp.loguniform('reg_lambda', np.log(1e-9), np.log(1)),
    'min_split_gain': hp.loguniform('min_split_gain', np.log(1e-9), np.log(1))
}

# Create Trials object to store optimization results
trials = Trials()

# Run hyperopt optimization
best3 = fmin(fn=objective, space=space3, algo=tpe.suggest, max_evals=200, trials=trials, rstate=np.random.default_rng(seed=30))
print(best3)

best_params3 = {
    'learning_rate': best3["learning_rate"],
    'n_estimators': int(best3['n_estimators']),
    'num_leaves': int(best3['num_leaves']),
    'max_depth': int(best3['max_depth']),
    'min_child_samples': int(best3['min_child_samples']),
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'subsample': best3['subsample'],
    'colsample_bytree': best3['colsample_bytree'],
    'reg_alpha': best3['reg_alpha'],
    'reg_lambda': best3['reg_lambda'],
    'min_split_gain': best3['min_split_gain']
}

# Print the best hyperparameters
print("Best Hyperparameters:")
print(best_params3)


In [ ]:
lg3 = LGBMRegressor(**best_params3, random_state=0, n_jobs = -1)

lg3.fit(X_tr3, y_tr_log3, early_stopping_rounds=100, eval_metric=custom_rmsle, eval_set=[(X_tr3, y_tr_log3), (X_val3, y_val_log3)])

y_test_pred_log3 = lg3.predict(X_test3)
y_test_pred3 = np.expm1(y_test_pred_log3)

rmsle_test = rmsle(np.expm1(y_test_log3), y_test_pred3)
print('Test RMSLE:', rmsle_test)
#Test RMSLE: 0.4747587094525547

In [80]:
y_pred1 = lg1.predict(X_test_eng1)
y_pred1=np.expm1(y_pred1)

y_pred2 = lg2.predict(X_test_eng2)
y_pred2=np.expm1(y_pred2)

y_pred3 = lg3.predict(X_test_eng3)
y_pred3=np.expm1(y_pred3)

In [81]:
test1["predict"]=y_pred1
test2["predict"]=y_pred2
test3["predict"]=y_pred3

In [82]:
X_test=pd.merge(test, test1[["ID","predict"]], how="left", on="ID" )
X_test["predict"]=X_test["predict"].fillna(0)
X_test=pd.merge(X_test, test2[["ID","predict"]], how="left", on="ID" )
X_test["predict_y"]=X_test["predict_y"].fillna(0)
X_test=pd.merge(X_test, test3[["ID","predict"]], how="left", on="ID" )
X_test["predict"]=X_test["predict"].fillna(0)
X_test["predict"]= X_test["predict_x"]+X_test["predict_y"]+X_test["predict"]

In [83]:
ss['ECLO'] = X_test["predict"]
ss.head()

,ID,ECLO
0,ACCIDENT_39609,3.883765
1,ACCIDENT_39610,3.931548
2,ACCIDENT_39611,5.307546
3,ACCIDENT_39612,4.674373
4,ACCIDENT_39613,4.801746


In [84]:
OUTPUT_PATH = './output/'
ss.to_csv(f'submission_bayesian.csv', index=False)